In [3]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import os
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import re

# Directory where the model is saved
model_dir = "./model"

# Load the model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = T5ForConditionalGeneration.from_pretrained(model_dir).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

# Function to extract video ID from YouTube URL
def extract_video_id(youtube_url):
    video_id_match = re.match(r'(https?://)?(www\.)?(youtube\.com|youtu\.?be)/.+((v=|\/)([^#\&\?]*))', youtube_url)
    if video_id_match:
        video_id = video_id_match.groups()[-1]
        return video_id
    return None

# Function to get transcript from YouTube video ID
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = ' '.join([item['text'] for item in transcript])
        return transcript_text
    except Exception as e:
        print(f"Error retrieving transcript: {e}")
        return None

# Function to clean the transcript text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove excessive whitespace
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Function to summarize text using the T5 model
def summarize_text(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=500, min_length=150, length_penalty=2.0, num_beams=5, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Function to summarize YouTube video by URL
def summarize_youtube_video(youtube_url):
    video_id = extract_video_id(youtube_url)
    if video_id:
        print(f"Extracted video ID: {video_id}")
        transcript = get_transcript(video_id)
        if transcript:
            print("Transcript retrieved successfully.")
            cleaned_transcript = clean_text(transcript)
            summary = summarize_text(cleaned_transcript)
            print("Summary:")
            print(summary)
        else:
            print("Failed to retrieve transcript.")
    else:
        print("Invalid YouTube URL.")

# User input for YouTube URL
youtube_url = input("Enter the YouTube video URL: ")
summarize_youtube_video(youtube_url)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Enter the YouTube video URL:  https://www.youtube.com/watch?v=_B6vtMa-US4


Extracted video ID: _B6vtMa-US4
Transcript retrieved successfully.
Summary:
the RTX 4060 TI supports all of nvidia's latest Ray tracing and AI upscaling bells and whistles it's much more power efficient than the card that it replaces. it supports all of nvidia's latest Ray tracing and AI upscaling bells and whistles it's much more power efficient than the card that it replaces and it supports av1 encoding which is a huge deal if you stream to YouTube. a... n a new nvidia's. and nvidia's latest. and a..
